<a href="https://colab.research.google.com/github/mjmantillav24/AnalisisEEG/blob/main/arquisoft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!pip install pyngrok
!ngrok authtoken 2uH83kbgWxd66wREnyRDfCvUIjH_RZyRKbmaUHsBhLzdKtPk




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
from pyngrok import ngrok

# Crear un túnel para el puerto 5000 donde corre Flask
public_url = ngrok.connect(5000)
print("Servidor accesible en:", public_url)


Servidor accesible en: NgrokTunnel: "https://e67d-35-245-50-18.ngrok-free.app" -> "http://localhost:5000"


In [15]:
!curl http://localhost:4040/api/tunnels





{"tunnels":[{"name":"http-5000-96d9005f-9dba-4901-9f79-b4aa69582721","ID":"33199bf8ee187f59932902f230d62012","uri":"/api/tunnels/http-5000-96d9005f-9dba-4901-9f79-b4aa69582721","public_url":"https://dde0-35-245-50-18.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"http-5000-d54e0e3c-09e8-49c4-ab68-15780dbdace8","ID":"bec33753bed8884777cf5cdc268218dd","uri":"/api/tunnels/http-5000-d54e0e3c-09e8-49c4-ab68-15780dbdace8","public_url":"https://e67d-35-245-50-18.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":1,"gauge":0,"rate1":0.00010774729930169493,"rate5":0.001216102466295947,"rate15":0.0007939384739658664,"p50":3773925,"p90":3773925,"p95":3773925,"p99":3773925},"http":{"count":1,"rate1

In [7]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 93.2 MB/s eta 0:00:00


In [16]:
from flask import Flask, request
import threading
import mne
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import os


app = Flask(__name__)


# 🔹 2. Definir la ruta del modelo en Google Drive
MODEL_PATH = "/content/drive/MyDrive/modelo_completo.pth"  # Ajusta la ruta según tu Drive

@app.route('/upload', methods=['POST'])
def upload_file():
    """Recibe un archivo desde la VM y lo guarda en Colab"""
    file = request.files['file']
    file_path = f"/content/{file.filename}"
    file.save(file_path)  # Guarda el archivo en Colab
     # 🔹 Hacer la predicción
    resultado = iniciar(file_path)  # Llama a la función de predicción
    return {"mensaje": resultado}, 200

def run_server():
    """Ejecuta el servidor en un hilo separado"""
    app.run(host='0.0.0.0', port=5000)

# Iniciar el servidor sin bloquear el notebook
threading.Thread(target=run_server).start()

# 🔹 Definir el modelo CNN
class EEG_CNN(nn.Module):
    def __init__(self, num_classes=4):
        super(EEG_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 5), padding=(1, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 5), padding=(1, 2))
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 5), padding=(1, 2))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten_size = 128 * 2 * 62
        self.fc1 = nn.Linear(self.flatten_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def preprocess(path: str):
    """Preprocesa el archivo EEG y lo convierte en tensor"""
    channels_to_remove = ['EEG Cz-Ref', 'EEG Pz-Ref', 'ECG EKG', 'Manual']
    data = mne.io.read_raw_edf(path, preload=True)
    channels = data.ch_names
    indices_to_remove = [i for i, ch in enumerate(channels) if ch in channels_to_remove]
    raw_data = np.array(data.get_data())

    while raw_data.shape[0] > 19 and indices_to_remove:
        raw_data = np.delete(raw_data, indices_to_remove.pop(), axis=0)

    if raw_data.shape[0] > 19:
        raw_data = raw_data[:19, :]
    elif raw_data.shape[0] < 19:
        raw_data = np.vstack((raw_data, np.zeros((19 - raw_data.shape[0], raw_data.shape[1]))))

    num_samples = raw_data.shape[1]
    segment_size = 500
    N = num_samples // segment_size
    segmented_data = raw_data[:, :N * segment_size].reshape(N, 19, 500)
    tensor_data = torch.tensor(segmented_data, dtype=torch.float32)
    return tensor_data

def predecir(modelo, entrada):
    """Hace la predicción con el modelo"""
    modelo.eval()
    with torch.no_grad():
        salida = modelo(entrada)
        prediccion = torch.argmax(salida, dim=1)
    return prediccion.item()

def iniciar(path: str):
    """Carga el modelo desde Drive, preprocesa y predice"""
    data = preprocess(path)

    # 🔹 Cargar el modelo desde Google Drive
    modelo_cargado = torch.load(MODEL_PATH, map_location=torch.device('cpu'))

    salida=[]
    for i in range(data.shape[0]):
      salida.append( predecir(modelo_cargado, data[i].unsqueeze(0).unsqueeze(0) ))

    if sum(salida)>0:
      return 'Se detecta epilepsia en EEG'
    else:
      return 'No se detecta epilepsia en EEG'



 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
